In [1]:
import math
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import MeanSquaredLogarithmicError
import os
import numpy as np

In [2]:
root = r'C:\Users\x\Desktop\vid_top\feat'
paths = [root+'/'+file for file in os.listdir(root)]

In [3]:
data = np.genfromtxt(paths[0],delimiter=',')
train_data = data[np.random.choice(len(data),1000)]
test_data = data[np.random.choice(len(data),100)]
for i in range(1,len(paths)):
    data = np.genfromtxt(paths[i],delimiter=',')
    train_data = np.concatenate([train_data,data[np.random.choice(len(data),1000)]])
    test_data = np.concatenate([test_data,data[np.random.choice(len(data),100)]])

In [12]:
class AutoEncoders(Model):
    def __init__(self, output_units):
        super().__init__()
        self.encoder = Sequential(
            [
                Dense(32, activation="relu"),
                Dense(16, activation="selu"),
                Dense(8, activation="selu")
            ]
        )
        self.decoder = Sequential(
            [
                Dense(16, activation="selu"),
                Dense(32, activation="selu"),
                Dense(output_units, activation="relu")
            ]
        )
    def call(self, inputs):
        encoded = self.encoder(inputs)
        decoded = self.decoder(encoded)
        return decoded
  
auto_encoder = AutoEncoders(len(train_data[0]))

auto_encoder.compile(
    loss='mae',
    metrics=['mae'],
    optimizer='adam'
)

history = auto_encoder.fit(
    train_data, 
    train_data, 
    epochs=500, 
    batch_size=32, 
    validation_data=(test_data,test_data)
)


Epoch 1/500
1438/1438 [==============================] - 6s 4ms/step - loss: 0.1143 - mae: 0.1143 - val_loss: 0.0826 - val_mae: 0.0826
Epoch 2/500
1438/1438 [==============================] - 5s 4ms/step - loss: 0.0812 - mae: 0.0812 - val_loss: 0.0789 - val_mae: 0.0789
Epoch 3/500
1438/1438 [==============================] - 5s 4ms/step - loss: 0.0784 - mae: 0.0784 - val_loss: 0.0767 - val_mae: 0.0767
Epoch 4/500
1438/1438 [==============================] - 5s 4ms/step - loss: 0.0768 - mae: 0.0768 - val_loss: 0.0756 - val_mae: 0.0756
Epoch 5/500
1438/1438 [==============================] - 5s 4ms/step - loss: 0.0670 - mae: 0.0670 - val_loss: 0.0650 - val_mae: 0.0650
Epoch 6/500
1438/1438 [==============================] - 5s 4ms/step - loss: 0.0649 - mae: 0.0649 - val_loss: 0.0636 - val_mae: 0.0636
Epoch 7/500
1438/1438 [==============================] - 5s 4ms/step - loss: 0.0639 - mae: 0.0639 - val_loss: 0.0644 - val_mae: 0.0644
Epoch 8/500
1438/1438 [==============================] 

In [13]:
auto_encoder.summary()

Model: "auto_encoders_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_8 (Sequential)    (None, 8)                 1848      
_________________________________________________________________
sequential_9 (Sequential)    (None, 36)                1876      
Total params: 3,724
Trainable params: 3,724
Non-trainable params: 0
_________________________________________________________________


In [15]:
encoder_layer = auto_encoder.get_layer('sequential_8')

In [16]:
data = np.genfromtxt(paths[0],delimiter=',')

In [17]:
auto_encoder.predict(data)[0]-data[0]

array([ 1.31921051e-02, -8.62096668e-03, -6.37488779e-03,  1.16611796e-02,
       -3.87586851e-03,  1.26467737e-02, -4.30945785e-03, -3.20269623e-02,
        3.98817677e-03, -2.35863229e-03,  9.14086183e-03, -1.80751307e-02,
        2.56382855e-02,  3.83840071e-04, -1.01968065e-02,  2.18670485e-02,
       -4.57370805e-03,  2.66838901e-02, -9.19890345e-03,  1.39683239e-02,
       -1.46805459e-02,  2.25100853e-02,  9.01458648e-03, -1.55371848e-05,
        2.94329914e-03, -1.68281569e-02,  3.01327029e-02, -2.19633774e-02,
        2.27381108e-02,  5.73219336e-03,  8.41538318e-03,  2.73797547e-02,
        3.67535677e-02,  1.74483941e-02,  3.40855362e-02, -1.66780544e-02])

In [18]:
encoder_layer.predict(data)[0]

array([ 0.15460056, -0.34659544, -0.62647206,  0.35517985, -0.16348143,
        0.36550122,  0.06164918, -0.6298809 ], dtype=float32)

In [19]:
for i in range(len(paths)):
    save_path = paths[i].replace('feat','feat_AE')
    data = np.genfromtxt(paths[i],delimiter=',')
    featAE = encoder_layer.predict(data)
    np.savetxt(save_path,featAE,delimiter=",")

In [22]:
# single file
path = r'C:\Users\x\Desktop\vid_top\0test\short_feat.csv'
save_path = r'C:\Users\x\Desktop\vid_top\0test\short_feat_AE.csv'
data = np.genfromtxt(path,delimiter=',')
featAE = encoder_layer.predict(data)
np.savetxt(save_path,featAE,delimiter=",")